In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import re
import nibabel as nib
import os
import sys
import time
import glob
import subprocess
import shutil
os.chdir('/local/home/dhaziza/entrack')
sys.path.append('/local/home/dhaziza/entrack/')

# PPMI DTI - manual inspection
weird_images = \
"""I466961_FA
I365105_FA
I643391_FA
I287984_FA
I344285_FA
I373013_FA""".split('\n')

In [ ]:
# Average images
def generate_average_image(glob_pattern, save_average_to):
    all_files = glob.glob(glob_pattern)
    all_correct_files = []
    total_image = np.zeros([116, 116, 74], dtype=np.float32)
    print('Computing average over %d images' % len(all_files))
    for f in all_files:
        skip = False
        for fs in weird_images:
            if fs in f:
                skip = True
        if skip:
            continue
        img = nib.load(f).get_data()
        if np.max(img) < 0.1:
            continue
        # Some images are [116, 116, 74], [116, 116, 72], [116, 106, 72], [122, 122, 72], [116, 116, 76], ...
        img_padded = np.zeros(total_image.shape)
        img_padded[:img.shape[0], :img.shape[1], :img.shape[2]] = img[:total_image.shape[0], :total_image.shape[1], :total_image.shape[2]]
        all_correct_files.append(f)
        total_image += img_padded

    average_image = total_image / float(len(all_correct_files))
    ref_file = nib.load(all_correct_files[0])
    new_img = nib.Nifti1Image(
        average_image,
        ref_file.affine,
        ref_file.header,
    )
    nib.save(new_img, save_average_to)

    plt.imshow(average_image[:, :, 36])
    plt.title('average_image[:, :, 36]')
    plt.show()

    plt.imshow(average_image[:, :, 50])
    plt.title('average_image[:, :, 50]')
    plt.show()
    return all_correct_files

def _exec(cmd):
    print('[Exec] ' + cmd)
    os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
    os.environ['FSLDIR'] = '/local/fsl'
    subprocess.call(cmd, shell=True)

def align_to_average(save_average_to, all_files_todo, to_dir):
    print('align_to_average: %d files to process' % len(all_files_todo))
    for f in all_files_todo:
        cmd = 'flirt -in {mri_image} -ref {ref} -out {out} ' + \
                '-cost {cost} -searchcost {searchcost} '
        cmd = cmd.format(
            mri_image=f,
            ref=save_average_to,
            out=os.path.join(to_dir, os.path.basename(f).split('_')[0] + '.nii.gz'),
            cost='normcorr',
            searchcost='normcorr',
        )
        _exec(cmd)

In [ ]:
FROM_DIR = '/local/PPMI/_dti_fitted'
TO_DIR = '/local/PPMI/_dti_fitted_fa_manual_filtered_aligned'
SAVE_AVERAGE_TO = '/local/PPMI/_dti_fitted_average.nii.gz'
GLOB_PATTERN = '%s/*_FA.nii.gz' % FROM_DIR

generate_average_image(GLOB_PATTERN, SAVE_AVERAGE_TO)
align_to_average(SAVE_AVERAGE_TO, all_correct_files, TO_DIR)

In [ ]:
# Second iteration
FROM_DIR = '/local/PPMI/_dti_fitted_fa_manual_filtered_aligned'
TO_DIR = '/local/PPMI/_dti_fitted_fa_manual_filtered_aligned_it2'
SAVE_AVERAGE_TO = '/local/PPMI/_dti_fitted_average_2.nii.gz'
GLOB_PATTERN = '%s/*.nii.gz' % FROM_DIR

generate_average_image(GLOB_PATTERN, SAVE_AVERAGE_TO)
align_to_average(SAVE_AVERAGE_TO, all_correct_files, TO_DIR)